In [ ]:
import pandas as pd
df_acl_24 = pd.read_csv("Dataset/df_acl_24_final.csv")
df_acl_23 = pd.read_csv("Dataset/df_acl_23_final.csv")
# # df_acl_23_final.csv

'Future Work' extraction

In [ ]:
import re

# Function to extract 'Future Work' and the next five sentences, with periods
def extract_future_work(text):
    # Split text into sentences
    sentences = text.split('.')
    # Iterate through sentences to find 'Future Work'
    for i, sentence in enumerate(sentences):
        if 'Future' in sentence:
            # Extract the sentence containing 'Future Work' and the next five sentences
            # Re-add the periods to each sentence as they are joined
            extracted_text = '.'.join(sentences[i:i+5]).strip()
            if not extracted_text.endswith('.'):
                extracted_text += '.'  # Ensure the extracted text ends with a period
            return extracted_text
    return ""  # Return an empty string if 'Future Work' is not found

# Apply the function to the DataFrame
df['Extracted Future Work'] = df['Concatenated Text'].apply(extract_future_work)


In [ ]:
# remove rest sentences if you find any word from 'keywords_to_check'

# Assuming df_acl_24 is already defined and 'Extracted Future Work' has been populated
keywords_to_check = [
    'acknowledgements','acknowledgments','acknowledgment','ethics', 'ethical', 'conclusion',
    'broader impact', 'discussion', 'limitations', 'appendix','acknowledgement'
]

# Function to remove sentences after encountering specific keywords
def remove_sentences_after_keywords(text):
    sentences = text.split('.')  # Split into sentences
    filtered_sentences = []
    for sentence in sentences:
        # Check if any keyword is in the current sentence
        if any(keyword.lower() in sentence.lower() for keyword in keywords_to_check):
            break  # Stop adding sentences if keyword is found
        filtered_sentences.append(sentence)
    # Join the sentences back into a single string with periods
    return '.'.join(filtered_sentences).strip() + '.'

# Apply the function to the 'Extracted Future Work' column
df['Extracted Future Work'] = df['Extracted Future Work'].apply(remove_sentences_after_keywords)


In [ ]:
# Drop rows where 'Extracted Future Work' is empty
df = df[df['Extracted Future Work'].notna() & (df['Extracted Future Work'] != "")]
# Reset the index of the DataFrame, dropping the old index
df.reset_index(drop=True, inplace=True)

In [ ]:
# Filter out rows where 'Extracted Future Work' is exactly '.'
df = df[df['Extracted Future Work'] != '.']

# Optionally, reset the index if you want a clean, consecutive index after removing rows
df.reset_index(drop=True, inplace=True)

In [ ]:
# Function to remove content within parentheses
def remove_parentheses(text):
    if isinstance(text, str):
        # Replace content within parentheses, including the parentheses
        return re.sub(r'\(.*?\)', '', text)
    return text  # Return the original text if it's not a string

# Apply the function to each element in the DataFrame
df = df.applymap(remove_parentheses)


<ipython-input-8-a241001aaf8e>:9: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(remove_parentheses)


In [ ]:
# convert to string
df = df.astype(str)

In [ ]:
# Convert Floats to Strings in All Columns:
df = df.applymap(lambda x: str(x) if isinstance(x, float) else x)

<ipython-input-10-ca3585b80a12>:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: str(x) if isinstance(x, float) else x)


In [ ]:
# remove '\n' charachter from 'all columns'

def clean_text(text):
    if pd.isna(text):
        return text
    sentences = text.split('.')
    cleaned_sentences = [sentence.replace('\n', ' ').strip() for sentence in sentences]
    return '. '.join(cleaned_sentences).strip()

for column in df.columns:
    if column in df.columns:
        df[column] = df[column].apply(clean_text)


measuring Average Cosine Similarity

In [ ]:
!pip -q install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.2/255.2 kB 4.1 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

model = SentenceTransformer('all-MiniLM-L6-v2')

def calculate_average_cosine_similarity(df, base_col, other_col):
    # Embed the text of the base column and the other column
    base_embeddings = model.encode(df[base_col].fillna("").tolist(), convert_to_tensor=True)
    other_embeddings = model.encode(df[other_col].fillna("").tolist(), convert_to_tensor=True)

    # Ensure both tensors are on CPU before calculating cosine similarity
    base_embeddings = base_embeddings.cpu()
    other_embeddings = other_embeddings.cpu()

    # Calculate cosine similarities and return the average
    similarities = cosine_similarity(base_embeddings, other_embeddings)
    return np.mean(similarities)

# Columns to exclude from comparison (e.g., non-text columns or the 'limitation' itself)
columns_to_exclude = ['Extracted Future Work']

# Store the results
average_similarities = {}

# Iterate over all columns except the 'limitation' column
for col in df.columns:
    if col not in columns_to_exclude:
        avg_sim = calculate_average_cosine_similarity(df, 'Extracted Future Work', col)
        average_similarities[col] = avg_sim

# Print average cosine similarities
print(average_similarities)


In [ ]:
import pandas as pd

# Assuming df is your DataFrame and 'Extracted Future Work' is the column of interest

# Function to calculate the number of words in a string
def word_count(text):
    words = text.split()  # Split the text into words based on whitespace
    return len(words)  # Return the number of words

# Apply the function to the 'Extracted Future Work' column to get word counts for each row
df['Word Count'] = df['Extracted Future Work'].apply(word_count)

# Calculate the average word count across all rows
average_word_count = df['Word Count'].mean()

# Print the average word count
print(f"Average Word Count: {average_word_count:.2f}")


In [ ]:
# Filter out rows where 'Word Count' is less than 10
df = df[df['Word Count'] >= 10]

# Optionally, reset the index if you want a clean, consecutive index after removing rows
df.reset_index(drop=True, inplace=True)

In [ ]:
df.to_csv('df_data_preprocessed.csv', index=False)